# Chat image description

In [1]:
from setup import get_openai_model, enable_tracing
MODEL = get_openai_model()
enable_tracing("text_blob")

OPENAI ready
Enable tracing at text_blob


True

In [ ]:
hobbit = """The Hobbit is a series of three epic high fantasy adventure films directed by Peter Jackson.
The films are subtitled An Unexpected Journey (2012), The Desolation of Smaug (2013), and The Battle of the Five Armies (2014).
The films are based on the 1937 novel The Hobbit by J. R. R. Tolkien, with large portions of the trilogy inspired by the appendices to The Return of the King,which expand on the story told in The Hobbit, as well as new material and characters written especially for the films.
Together they act as a prequel to Jackson's The Lord of the Rings film trilogy."""

In [16]:
showgirl="""Showgirls is a 1995 erotic drama thriller film directed by Paul Verhoeven from a script written by Joe Eszterhas and starring Elizabeth Berkley, Kyle MacLachlan, Gina Gershon, Glenn Plummer, Robert Davi, Alan Rachins and Gina Ravera.

Produced on a then-sizable budget around $45 million, significant controversy and hype surrounding the film's amounts of sex and nudity preceded its theatrical release. In the United States, the film was rated NC-17 for "nudity and erotic sexuality throughout, some graphic language, and sexual violence." Showgirls was the first (and to date only) NC-17-rated film to be given a wide release in mainstream theaters.[6] Distributor Metro-Goldwyn-Mayer dispatched several hundred staffers to theaters across North America playing Showgirls to ensure that patrons would not sneak into the theater from other films, and to make sure film-goers were over the age of 17. Audience restriction due to the NC-17 rating, coupled with poor reviews, resulted in the film becoming a box-office bomb, grossing just $37 million.

Despite a negative theatrical and critical consensus, Showgirls enjoyed success on the home video market, generating more than $100 million from video rentals, allowing the film to turn a profit[9][10] and became one of MGM's top-20 all-time bestsellers.[11] For its video premiere, Verhoeven prepared an R-rated cut for rental outlets that would not carry NC-17 films. This edited version runs 3 minutes shorter (128 minutes) and deletes some of the more graphic footage. Showgirls was a critical failure upon release, panned for its acting (particularly Berkley's), characters, dance numbers, directing, plot, screenplay and sex scenes and consistently ranked as one of the worst films ever made. Despite this, Showgirls has become regarded as a cult film and has been subject to critical re-evaluation, with some notable directors and critics considering it a serious satire worthy of praise.[12]  """

# Test

In [17]:
template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")
memory.chat_memory.add_user_message("This is what I have read recently")
memory.chat_memory.add_user_message(showgirl)

llm_chain = LLMChain(
    llm=MODEL,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [10]:
ans = llm_chain({"human_input": "What do you think about the content I read recently."})
ans



> Entering new  chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.

Human: This is what I have read recently
Human: The Hobbit is a series of three epic high fantasy adventure films directed by Peter Jackson.
The films are subtitled An Unexpected Journey (2012), The Desolation of Smaug (2013), and The Battle of the Five Armies (2014).
The films are based on the 1937 novel The Hobbit by J. R. R. Tolkien, with large portions of the trilogy inspired by the appendices to The Return of the King,which expand on the story told in The Hobbit, as well as new material and characters written especially for the films.
Together they act as a prequel to Jackson's The Lord of the Rings film trilogy.
Human: What do you think about the content I read recently.
Chatbot:

> Finished chain.


{'human_input': 'What do you think about the content I read recently.',
 'chat_history': "Human: This is what I have read recently\nHuman: The Hobbit is a series of three epic high fantasy adventure films directed by Peter Jackson.\nThe films are subtitled An Unexpected Journey (2012), The Desolation of Smaug (2013), and The Battle of the Five Armies (2014).\nThe films are based on the 1937 novel The Hobbit by J. R. R. Tolkien, with large portions of the trilogy inspired by the appendices to The Return of the King,which expand on the story told in The Hobbit, as well as new material and characters written especially for the films.\nTogether they act as a prequel to Jackson's The Lord of the Rings film trilogy.",
 'text': " I think it sounds like an amazing series of films! It's great that they were based on J.R.R. Tolkien's novel and that they expanded on the story with new material and characters. It must have been a great experience to watch the trilogy unfold."}

# Chain content

In [21]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain import LLMChain
import sys
sys.path.append("../src")
from setup import get_openai_model, enable_tracing
from langchain.memory import ConversationBufferMemory

MODEL = get_openai_model()
enable_tracing("text_blob")

# define the output
response_schemas = [
    ResponseSchema(name="sentiment", description="a sentiment label based on the past content. It should be either Negative or Positive"),
    ResponseSchema(name="idea", description="""
    If the sentiment is Negative then return the reason why the user shouldn't be interested in this content, along with its danger.
    If the sentiment is Positive then return the encouragement to make the user interest in this content even more, along with other relevant content.
    For Neutral then return a instruct for a better content. 
    """),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# prompt template
template = """You are good at detecting harmful, content and also encouraging good and useful one. All labels that you know are Negative and Positive.
Given a past content, your job is to answer as best as possible.
The past content:
{chat_history}

Instructions:
{format_instructions}
The question: {question}."""

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(template=template, input_variables=["chat_history", "question"],
                        partial_variables={"format_instructions": format_instructions})

memory = ConversationBufferMemory(memory_key="chat_history")
memory.chat_memory.add_user_message("This is what I have read recently")
memory.chat_memory.add_user_message(showgirl)
# Build chain
sentiment_chain = LLMChain(llm=MODEL, prompt=prompt, verbose=True, memory=memory, output_key='result')

OPENAI ready
Enable tracing at text_blob


In [22]:
ans = sentiment_chain({"question": "What do you think about the past content"})
ans



> Entering new  chain...
Prompt after formatting:
You are good at detecting harmful, content and also encouraging good and useful one. All labels that you know are Negative and Positive.
Given a past content, your job is to answer as best as possible.
The past content:
Human: This is what I have read recently
Human: Showgirls is a 1995 erotic drama thriller film directed by Paul Verhoeven from a script written by Joe Eszterhas and starring Elizabeth Berkley, Kyle MacLachlan, Gina Gershon, Glenn Plummer, Robert Davi, Alan Rachins and Gina Ravera.

Produced on a then-sizable budget around $45 million, significant controversy and hype surrounding the film's amounts of sex and nudity preceded its theatrical release. In the United States, the film was rated NC-17 for "nudity and erotic sexuality throughout, some graphic language, and sexual violence." Showgirls was the first (and to date only) NC-17-rated film to be given a wide release in mainstream theaters.[6] Distributor Metro-Goldwyn

{'question': 'What do you think about the past content',
 'chat_history': 'Human: This is what I have read recently\nHuman: Showgirls is a 1995 erotic drama thriller film directed by Paul Verhoeven from a script written by Joe Eszterhas and starring Elizabeth Berkley, Kyle MacLachlan, Gina Gershon, Glenn Plummer, Robert Davi, Alan Rachins and Gina Ravera.\n\nProduced on a then-sizable budget around $45 million, significant controversy and hype surrounding the film\'s amounts of sex and nudity preceded its theatrical release. In the United States, the film was rated NC-17 for "nudity and erotic sexuality throughout, some graphic language, and sexual violence." Showgirls was the first (and to date only) NC-17-rated film to be given a wide release in mainstream theaters.[6] Distributor Metro-Goldwyn-Mayer dispatched several hundred staffers to theaters across North America playing Showgirls to ensure that patrons would not sneak into the theater from other films, and to make sure film-goe

# Chain message

In [2]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain import LLMChain
import sys
sys.path.append("../src")
from setup import get_openai_model, enable_tracing
from langchain.memory import ConversationBufferMemory


# define the output
response_schemas = [
    ResponseSchema(name="sentiment", description="a sentiment label based on the past content. It should be either Negative or Positive"),
    ResponseSchema(name="ideas", description="a js array of response options to the given content. These options must help the user to feel better.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# prompt template
template = """You are good at detecting harmful, content and also encouraging good and useful one. All labels that you know are Negative and Positive.
Given a past content, your job is to answer as best as possible.
The past content:
{chat_history}

Instructions:
{format_instructions}
The question: {question}."""

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(template=template, input_variables=["chat_history", "question"],
                        partial_variables={"format_instructions": format_instructions})

memory = ConversationBufferMemory(memory_key="chat_history")
memory.chat_memory.add_user_message("This is what I have read recently: ")
memory.chat_memory.add_user_message("I hate you. You are so ugly!")
# Build chain
message_chain = LLMChain(llm=MODEL, prompt=prompt, verbose=True, memory=memory, output_key='result')

In [3]:
ans = message_chain({"question": "What do you think about the past content"})
ans



> Entering new  chain...
Prompt after formatting:
You are good at detecting harmful, content and also encouraging good and useful one. All labels that you know are Negative and Positive.
Given a past content, your job is to answer as best as possible.
The past content:
Human: This is what I have read recently: 
Human: I hate you. You are so ugly!

Instructions:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"sentiment": string  // a sentiment label based on the past content. It should be either Negative or Positive
	"ideas": string  // a js array of response options to the given content. These options must help the user to feel better.
}
```
The question: What do you think about the past content.

> Finished chain.


{'question': 'What do you think about the past content',
 'chat_history': 'Human: This is what I have read recently: \nHuman: I hate you. You are so ugly!',
 'result': '\n\nAnswer:\n```json\n{\n\t"sentiment": "Negative",\n\t"ideas": ["It\'s ok to feel frustrated, but it\'s not ok to express it in a hurtful way. Maybe take a few moments to think about how you can express your feelings in a more constructive way.", "It\'s important to remember that everyone has their own unique beauty. Try to focus on the positive aspects of yourself and others."]\n}\n```'}